In [1]:
import xarray as xr
import dask
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import glob
import pandas as pd
import os
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import zarr


#  Find the list of variables
##  1.  Get a full list of variables in the hourly data

In [3]:
one_hourly_file = '/tmpdata/i.e215.I2000Clm50SpGs.hw_production.02/i.e215.I2000Clm50SpGs.hw_production.02.clm2.h2.1986-01-01-00000.nc'
ds_hourly = xr.open_dataset(one_hourly_file)
ds_hourly

<xarray.Dataset>
Dimensions:        (levgrnd: 25, levlak: 10, levdcmp: 1, time: 24,
                    hist_interval: 2, lon: 288, lat: 192)
Coordinates:
  * levgrnd        (levgrnd) float32 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
  * levlak         (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.33 44.78
  * levdcmp        (levdcmp) float32 1.0
  * time           (time) object 1986-01-01 00:00:00 ... 1986-01-01 23:00:00....
  * lon            (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lat            (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
Dimensions without coordinates: hist_interval
Data variables: (12/65)
    mcdate         (time) int32 ...
    mcsec          (time) int32 ...
    mdcur          (time) int32 ...
    mscur          (time) int32 ...
    nstep          (time) int32 ...
    time_bounds    (time, hist_interval) object ...
    ...             ...
    VAPOR_PRES_U   (time, lat, lon) float32 ...
    WASTEHEAT      (time, lat, lon) float32 ...
    WBA            (time, lat, lon) float32 ...
    WBA_R          (time, lat, lon) float32 ...
    WBA_U          (time, lat, lon) float32 ...
    WIND           (time, lat, lon) float32 ...
Attributes: (12/40)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 02/13/24 22:44:22
    source:                                    Community Land Model CLM4.0
    hostname:                                  JohnnyThinkStation
    ...                                        ...
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          hour_1
    Time_constant_3Dvars_filename:             ./i.e215.I2000Clm50SpGs.hw_pro...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

In [2]:
ds_hourly_vars = list(ds_hourly.data_vars)
ds_hourly_vars

['mcdate',
 'mcsec',
 'mdcur',
 'mscur',
 'nstep',
 'time_bounds',
 'date_written',
 'time_written',
 'area',
 'landfrac',
 'landmask',
 'pftmask',
 'nbedrock',
 'APPAR_TEMP',
 'APPAR_TEMP_R',
 'APPAR_TEMP_U',
 'EFLX_LH_TOT',
 'EFLX_LH_TOT_R',
 'EFLX_LH_TOT_U',
 'FGR',
 'FGR_R',
 'FGR_U',
 'FIRA',
 'FIRA_R',
 'FIRA_U',
 'FIRE',
 'FIRE_R',
 'FIRE_U',
 'FLDS',
 'FSA',
 'FSA_R',
 'FSA_U',
 'FSDS',
 'FSH',
 'FSH_R',
 'FSH_U',
 'HEAT_FROM_AC',
 'HIA',
 'HIA_R',
 'HIA_U',
 'PBOT',
 'Q2M',
 'Q2M_R',
 'Q2M_U',
 'QBOT',
 'RAIN',
 'SNOW',
 'TBOT',
 'THBOT',
 'TSA',
 'TSA_R',
 'TSA_U',
 'TSKIN',
 'TSKIN_R',
 'TSKIN_U',
 'U10',
 'URBAN_HEAT',
 'VAPOR_PRES',
 'VAPOR_PRES_R',
 'VAPOR_PRES_U',
 'WASTEHEAT',
 'WBA',
 'WBA_R',
 'WBA_U',
 'WIND']

##  2.  Define the core daily variables for UHI analysis

In [5]:
core_vars = ['TSA', 'TSA_R', 'TSA_U', 'WBA', 'WBA_R', 'WBA_U']
#all vars in ds_hourly_vars not in core_vars are other_vars
other_vars = [var for var in ds_hourly_vars if var not in core_vars]
other_vars

['mcdate',
 'mcsec',
 'mdcur',
 'mscur',
 'nstep',
 'time_bounds',
 'date_written',
 'time_written',
 'area',
 'landfrac',
 'landmask',
 'pftmask',
 'nbedrock',
 'APPAR_TEMP',
 'APPAR_TEMP_R',
 'APPAR_TEMP_U',
 'EFLX_LH_TOT',
 'EFLX_LH_TOT_R',
 'EFLX_LH_TOT_U',
 'FGR',
 'FGR_R',
 'FGR_U',
 'FIRA',
 'FIRA_R',
 'FIRA_U',
 'FIRE',
 'FIRE_R',
 'FIRE_U',
 'FLDS',
 'FSA',
 'FSA_R',
 'FSA_U',
 'FSDS',
 'FSH',
 'FSH_R',
 'FSH_U',
 'HEAT_FROM_AC',
 'HIA',
 'HIA_R',
 'HIA_U',
 'PBOT',
 'Q2M',
 'Q2M_R',
 'Q2M_U',
 'QBOT',
 'RAIN',
 'SNOW',
 'TBOT',
 'THBOT',
 'TSKIN',
 'TSKIN_R',
 'TSKIN_U',
 'U10',
 'URBAN_HEAT',
 'VAPOR_PRES',
 'VAPOR_PRES_R',
 'VAPOR_PRES_U',
 'WASTEHEAT',
 'WIND']